In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [2]:

# Load and preprocess Heart Disease dataset
#url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
columns = ['c1','c2','c3','c4','target']
data = pd.read_csv(url, header=None, names=columns, na_values="?")
data = data.dropna()  # Remove rows with missing values
data['target'] = (data['target'] > 0).astype(int)  # Convert to binary classification

In [3]:
X = data.drop(columns=['target']).values
y = data['target'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)



In [9]:
X_train_tensor

tensor([[-0.6412,  1.8034, -0.1640, -3.0268],
        [ 0.8237,  0.8412, -0.5721,  0.6033],
        [-1.6618, -1.6679,  2.4234, -0.3362],
        ...,
        [-2.6287,  1.2563, -0.2859, -1.5941],
        [-1.3704,  0.1309, -0.2869,  0.0806],
        [-1.3344, -1.5618,  2.3313,  0.2955]])

In [ ]:
# Define MLP model in PyTorch
class MLPModel(nn.Module):
    def __init__(self, input_dim):
        super(MLPModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64), # here only the parameters are also initialized along with layer initializationx
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)



In [ ]:
# Initialize model, loss function, and optimizer
input_dim = X.shape[1]
model = MLPModel(input_dim)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss - (-)summation yilog(yi) + (1-yi)log(1-yi)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# You're not initializing the parameters here - you're just telling the optimizer which already-initialized parameters it should update during training.

# model.parameters() returns all the learnable parameters (weights and biases) in your neural network. 
# These are what actually need to be adjusted during training.

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train() # the model is already loaded as the MLPModel which we defined earlier as the MLP model of sequential layers, 
                    #giving the input dimension to it will help train our model, as the parameters are already sent through the optimizer
    epoch_loss = 0.0
    correct = 0
    total = 0

    for batch_X, batch_y in train_loader: # the train loader has already loaded the datasets in batches for computational and memory efficiency
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y) # between the predicted and the actual results
        loss.backward() # backpropagation - the gradients are calculated and stored in the parameters
        optimizer.step() # the optimizer updates the parameters based on the gradients - here since the optimizer was Adam, 
                            #it will also take care of the collected gradients till now in its memory to update the parameters

        epoch_loss += loss.item()
        predictions = (outputs > 0.5).float()
        correct += (predictions == batch_y).sum().item()
        total += batch_y.size(0)

    epoch_accuracy = correct / total
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}, Accuracy: {epoch_accuracy:.4f}")


Epoch 1/10, Loss: 0.0964, Accuracy: 0.9599
Epoch 2/10, Loss: 0.0035, Accuracy: 1.0000
Epoch 3/10, Loss: 0.0232, Accuracy: 0.9936
Epoch 4/10, Loss: 0.0011, Accuracy: 1.0000
Epoch 5/10, Loss: 0.0002, Accuracy: 1.0000
Epoch 6/10, Loss: 0.0001, Accuracy: 1.0000
Epoch 7/10, Loss: 0.0001, Accuracy: 1.0000
Epoch 8/10, Loss: 0.0000, Accuracy: 1.0000
Epoch 9/10, Loss: 0.0000, Accuracy: 1.0000
Epoch 10/10, Loss: 0.0000, Accuracy: 1.0000


In [ ]:
# Evaluate the model - using the weights and biases that were updated during training (from the last epoch)
model.eval() #Sets a flag that tells all layers they're in evaluation mode
#Disables dropout (all neurons become active) 
with torch.no_grad(): # so that gradients are not tracked during evaluation
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor).item()
    test_predictions = (test_outputs > 0.5).float()
    test_accuracy = (test_predictions == y_test_tensor).float().mean().item()

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.0000, Test Accuracy: 1.0000


## Some ways to improve the performance of models MLP -
1. Experiment with the architecture structure, add batchnorm and drop out layers , try with more relevant activation functions
2. K fold cross validation for more robust evaluation
3. Ensemble methods
4. Data Augmentation 
